# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 11:23AM,243151,15,30009740,"Alliance Pharma, Inc.",Released
1,Aug 1 2022 11:23AM,243151,15,30009741,"Alliance Pharma, Inc.",Released
2,Aug 1 2022 11:23AM,243151,15,30009743,"Alliance Pharma, Inc.",Released
3,Aug 1 2022 11:23AM,243151,15,30009742,"Alliance Pharma, Inc.",Released
4,Aug 1 2022 11:23AM,243151,15,30009745,"Alliance Pharma, Inc.",Released
5,Aug 1 2022 11:23AM,243151,15,30009744,"Alliance Pharma, Inc.",Released
6,Aug 1 2022 11:23AM,243151,15,30009746,"Alliance Pharma, Inc.",Released
7,Aug 1 2022 11:23AM,243151,15,30009748,"Alliance Pharma, Inc.",Released
8,Aug 1 2022 11:23AM,243151,15,30009750,"Alliance Pharma, Inc.",Released
9,Aug 1 2022 11:23AM,243151,15,30009747,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,243145,Released,48
28,243146,Released,1
29,243147,Released,6
30,243150,Released,49
31,243151,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243145,NaN,48.0
243146,NaN,1.0
243147,NaN,6.0
243150,NaN,49.0
243151,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243085,0.0,1.0
243091,0.0,1.0
243101,0.0,1.0
243108,26.0,6.0
243110,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243085,0,1
243091,0,1
243101,0,1
243108,26,6
243110,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243085,0,1
1,243091,0,1
2,243101,0,1
3,243108,26,6
4,243110,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243085,,1
1,243091,,1
2,243101,,1
3,243108,26,6
4,243110,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc."
20,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation
69,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC"
95,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC"
97,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc."
103,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc."
104,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation
168,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc."
216,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc."
217,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc.",,20
1,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,,49
2,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",,26
3,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",,2
4,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",,6
5,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc.",,1
6,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,,64
7,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",,48
8,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",,1
9,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc.",20,
1,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,49,
2,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",26,
3,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",2,
4,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",6,
5,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc.",1,
6,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,64,
7,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",48,
8,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",1,
9,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc.",20,
1,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,49,
2,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",26,
3,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",2,
4,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc.",20.0,NaN
1,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,49.0,NaN
2,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",26.0,NaN
3,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN
4,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 11:23AM,243151,15,"Alliance Pharma, Inc.",20.0,0.0
1,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,49.0,0.0
2,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",26.0,0.0
3,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0
4,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3403790,319.0,7.0
15,729418,48.0,0.0
16,243123,2.0,0.0
19,1215651,9.0,27.0
20,1458707,58.0,7.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3403790,319.0,7.0
1,15,729418,48.0,0.0
2,16,243123,2.0,0.0
3,19,1215651,9.0,27.0
4,20,1458707,58.0,7.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,319.0,7.0
1,15,48.0,0.0
2,16,2.0,0.0
3,19,9.0,27.0
4,20,58.0,7.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,319.0
1,15,Released,48.0
2,16,Released,2.0
3,19,Released,9.0
4,20,Released,58.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,7.0,0.0,0.0,27.0,7.0
Released,319.0,48.0,2.0,9.0,58.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,7.0,0.0,0.0,27.0,7.0
1,Released,319.0,48.0,2.0,9.0,58.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,7.0,0.0,0.0,27.0,7.0
1,Released,319.0,48.0,2.0,9.0,58.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()